In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib import rc
from matplotlib.colors import LinearSegmentedColormap as colormap
from IPython.display import HTML
import functions as f
import os, sys
sys.path.append(os.path.relpath("./src"))
from kitaev_chain import kitaev_chain_model, quench_simulation_bdg, quench_simulation_tfim
from utils import to_n, s0, sx, sy, sz, power_set, tensor_product, canon_eigen, expm

## Find $\ket{0}$

In [ ]:
def c(i, n, dagger=False):
    As = [sz if j < i else 0.5 * (sx + (1 if dagger else -1) * 1j * sy) if j == i else s0 for j in range(n)]
    return tensor_product(As)


## Find $U, V$

In [ ]:
k = 1e-2
n = 8
N = 2 ** n
zero = np.zeros((2, 1))
zero[1][0] = 1
zero = tensor_product([zero] * n)

H0 = kitaev_chain_model(n, 0 + k, 1 - k, 1).bdg_hamiltonian()
evals, evecs = np.linalg.eigh(H0)
evals_sorted, evecs_sorted = canon_eigen(evals, evecs)

P = evecs_sorted.T
np.allclose(evecs_sorted @ np.diag(evals_sorted) @ evecs_sorted.T, H0) #sanity

In [ ]:
plt.plot(evals_sorted, '*')

In [ ]:
U = P[:n, :n]
Us = P[n:, n:]
V = P[:n, n:]
Vs = P[n:, :n]
np.allclose(U.T @ V + V.T @ U, U * 0) and np.allclose(U.T @ U + V.T @ V, np.eye(n)) and np.allclose(U, Us.conj()) and np.allclose(V, Vs.conj()) 

In [ ]:
idx = np.where(evals_sorted > 1e-12)[0][0] # first positive eigenvalue
U_prime = U[idx:, idx:]
V_prime = V[idx:, idx:]
G = -np.linalg.inv(U_prime) @ V_prime
A = 0.5 * sum([sum([G[i - idx][j - idx] * c(i, n, True) @ c(j, n, True) for i in range(idx, n)], np.zeros((N, N))) for j in range(idx, n)], np.zeros((N, N)))
vac = expm(A, 4) @ zero

In [ ]:
def psi(i, n):
    return sum([c(j, n) * U[i][j] + c(j, n, True) * V[i][j] for j in range(n)], np.zeros((N, N)))

In [ ]:
norms = [np.linalg.norm(psi(i, n) @ vac) for i in range(n)]

In [ ]:
plt.plot(norms)
